In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
%config Completer.use_jedi = False

In [2]:
#customsItemId = 29581
customsItemId = 32514

In [3]:
url = "https://shaarolami-query.customs.mof.gov.il/CustomspilotWeb/he/CustomsBook/Import/ImportCustomsItemDetails?customsItemId=" + str(customsItemId)


In [4]:
url


'https://shaarolami-query.customs.mof.gov.il/CustomspilotWeb/he/CustomsBook/Import/ImportCustomsItemDetails?customsItemId=32514'

In [5]:
page = urlopen(url)
html_bytes = page.read()
html = html_bytes.decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [6]:
title_index = html.find("<title>")
title_index

145

In [7]:
soup.find_all("title")

[<title> פרטי פרט מכס</title>]

In [8]:
soup.title

<title> פרטי פרט מכס</title>

In [9]:
# customItems of all parents of '2009111910/5'
items = soup.find_all("span", class_="treeTitle")
for item in items:
    #print(item.parent.name)
    print(item.text.strip())

2009000000
2009100000
2009110000
2009111000
2009111900
2009111910/5


In [10]:
# most detailed customItem ('2009111910/5': 10 digits + slash + additional digit)
item = soup.find_all("span", class_="treeTitle")[-1]
customItem = item.text.strip()
customItem

'2009111910/5'

In [11]:
# Hebrew description of the most specific customItem
# (This does not fully describes the item! you usually need Hebrew descriptions of parents to understand what is the item)
itemParent = item.parent.parent.find("div", style="display:none")
customItemLabel = itemParent.text.strip()
customItemLabel

'המשמש לייצור משקה שמוטל עליו מס, ובלבד שישמש כאמור וישולם  מס על המשקה (מותנה )'

In [12]:
# The customItemID of the item (used in the URL, and also in SQL tables)
items = soup.find_all("li")
last = items[-1]
itemId = last.attrs["id"]
itemId

'32514'

## scraping the part of 'דרישות חוקיות' in the page of this customItem

In [13]:
drishotTable = soup.find("table", class_="legalDemandsTable")
drishotBody = drishotTable.find("tbody")
len(drishotBody.find_all("tr"))


2

## retrieve regulators

In [14]:
trs = drishotBody.find_all("tr")
for tr in trs:
    tdx = tr.find_all("td")[4]
    print(tdx.text.strip())

משרד הבריאות
משרד הבריאות


## retrieve אישורים

In [31]:
def findTd(tr, index):
    return "'" + tr.find_all("td")[index].text.strip() + "'"

trs = drishotBody.find_all("tr")
for tr in trs:
#    print(findTd(tr, 1), findTd(tr, 2), findTd(tr, 3), findTd(tr, 4), findTd(tr, 5),
#          findTd(tr, 6), findTd(tr, 7), findTd(tr, 8), findTd(tr, 9), findTd(tr, 10))
    print(findTd(tr, 1), findTd(tr, 2), findTd(tr, 3), findTd(tr, 4), findTd(tr, 5))

'תוספת 2 לצו יבוא חופשי' '‎2000000000' 'למעט 20096000לא כולל 2009600000,' 'משרד הבריאות' 'אישור שחרור תחנת מעבר (0705) -'
'תוספת 2 לצו יבוא חופשי' '‎2000000000' 'למעט 20096000לא כולל 2009600000,' 'משרד הבריאות' 'אישור שירות המזון (0701) -'


## scraping the part of 'הנחיות סיווג' in the page of this customItem

In [33]:
# guidanceTable
sivugTable = soup.find("table", class_="guidanceTable")
body = sivugTable.find("tbody")
len(body.find_all("tr"))

2

In [27]:
def findTd(tr, index):
    return "'" + tr.find_all("td")[index].text.strip() + "'"

trs = body.find_all("tr")
for tr in trs:
    #td1 = findTd(tr, 1)
    print(findTd(tr, 1), findTd(tr, 2), findTd(tr, 3), findTd(tr, 4), findTd(tr, 5))


'001/13' '20.09 מיץ אלו-ורה ((Aloe vera juice' 'הנחיה לפרט מכס' '‎2009000000' '01/03/2013'
'001/22' '"ליים" ו"לימון"' 'הנחיה לפרט מכס' '‎2009000000' '08/02/2022'


## scraping the part of 'תדפיסי חקיקה מקחים' in the page of this customItem

In [34]:
# makachTable
makachTable = soup.find("table", class_="mekachTable")
body = makachTable.find("tbody")
len(body.find_all("tr"))

1

In [78]:
def removeUnPrintableCaracters(str):
    if str.isprintable():
        return str
    ret = ""
    for x in str:
        if (x.isprintable()):
            ret = ret + x
        else:
            ret = ret + '~'
    return ret

    
trs = body.find_all("tr")
for tr in trs:
    print(findTd(tr, 1), findTd(tr, 2), removeUnPrintableCaracters(findTd(tr, 3)), findTd(tr, 4))

'0' '' '01/01/2022~~שבת כ"ח טבת תשפ"ב' 'משקאות פרטים מותנים'


## a function to scrape any customItemId

In [97]:
def findPageByUrl(url):
    page = urlopen(url)
    html_bytes = page.read()
    html = html_bytes.decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    return soup
    
def scrapeAll(customsItemId):
    url = "https://shaarolami-query.customs.mof.gov.il/CustomspilotWeb/he/CustomsBook/Import/ImportCustomsItemDetails?customsItemId=" + str(customsItemId)
    #url = url + '&validToDate=12%2F31%2F2021%2000%3A00%3A00'
    #page = urlopen(url)
    #html_bytes = page.read()
    #html = html_bytes.decode("utf-8")
    #soup = BeautifulSoup(html, "html.parser")
    soup = findPageByUrl(url)

    # most detailed customItem ('2009111910/5': 10 digits + slash + additional digit)
    items = soup.find_all("span", class_="treeTitle")
    print(customsItemId, ' ##')
    if (len(items) == 0):
        print('add validDate')
        soup = findPageByUrl(url + '&validToDate=12%2F31%2F2021%2000%3A00%3A00')
        items = soup.find_all("span", class_="treeTitle")
    if (len(items) == 0):
#        print(customsItemId, ' ##')
        return
    item = soup.find_all("span", class_="treeTitle")[-1]
    customItem = item.text.strip()

    drishotTable = soup.find("table", class_="legalDemandsTable")
    if drishotTable:
        drishotBody = drishotTable.find("tbody")
        trs = drishotBody.find_all("tr")
        for tr in trs:
#    print(findTd(tr, 1), findTd(tr, 2), findTd(tr, 3), findTd(tr, 4), findTd(tr, 5),
#          findTd(tr, 6), findTd(tr, 7), findTd(tr, 8), findTd(tr, 9), findTd(tr, 10))
            print(findTd(tr, 1), findTd(tr, 2), findTd(tr, 3), findTd(tr, 4), findTd(tr, 5))
    else:
        print('no Drishot')

#scrapeAll(32514)
#scrapeAll(29861)
#scrapeAll(27189)
#scrapeAll(32514)

list = [3460,
        32524, #32512, 32513, 32514, 32515, 29861, 
        27189]
for id in list:
    scrapeAll(id)

3460  ##
'תוספת 2 לצו יבוא חופשי' '‎8415109000/2' 'אלה שתפוקת הקירור שלהם אינה עולה על 18 קו"ט' 'משרד האנרגיה' 'אישור אגף ניהול משאבי תשתיות (1001)                                                     
                                                        (
                                                            
                                                                אישור אגף ניהול משאבי תשתיות (1001) - מקררים
                                                            
                                                                , 
                                                            
                                                                אישור אגף ניהול משאבי תשתיות (1001) - מזגנים
                                                            
                                                                , 
                                                            
                                                                אישור אגף ניהול משאב